# Kirchhoff-Love plate with Hellan-Herrmann-Johnson method


For very thin plates like metal sheets, the Kirchhoff-Love hypothesis is frequently assumed, which states that normals of the original plate stay perpendicular to the mid-surface of the deformed plate. This means that no shearing occurs, i.e. the rotations $\beta$ from the Reissner-Mindlin plate equations can be eliminated by the gradient of the vertical deflection $w$ of the mid-surface. Note that $\nabla w$ is the linearization of the rotated normal vector of the plate. Eliminating $\beta$ in the Reissner-Mindlin plate equations leads to the Kirchhoff-Love plate equation, which is the form of a biharmonic problem

\begin{align*}
\int_{\Omega}\mathbb{D}\nabla^2 w:\nabla\delta w\,dx = \int_{\Omega}f\,\delta w\,dx,
\end{align*}

where $\mathbb{D}\varepsilon=\frac{Et^3}{12(1-\nu^2)}((1-\nu)\varepsilon+\nu\,\mathrm{tr}(\varepsilon)I_{2\times 2})$. The thickness parameter $t$ gets sometimes absorbed by the right-hand side $f$ by dividing the equation by $t^3$.

The problem is well-posed for $w\in H^2(\Omega)$ (Lax-Milgram) and reads in strong form
\begin{align*}
\mathrm{div}\mathrm{div}(\mathbb{D}\nabla^2w)=f.
\end{align*}
Due to the increased regularity of $w$ point forces $f$ are well-defined for dimensions 2 and 3 as $H^2(\Omega)\hookrightarrow C^0(\Omega)$. To solve the Kirchhoff-Love plate equation with a conforming Galerkin method in the elliptic setting would require $H^2$-conforming finite elements, where the derivatives are also continuous over elements. 

To solve the Kirchhoff-Love plate equation with a conforming Galerkin method in the elliptic setting would require $H^2$-conforming finite elements, where the derivatives are also continuous over elements. Also, (hybrid) Discontinuous-Galerkin methods [NGSolve docu - Fourth order equation](https://docu.ngsolve.org/nightly/i-tutorials/unit-2.9-fourthorder/fourthorder.html) are possible, but would need a high polynomial degree.

## Hellan-Herrmann-Johnson method
Instead, we use the Hellan-Herrmann-Johnson (HHJ) method for Kirchhoff-Love plates. After some decades, the method regained huge interest. See e.g. [Hellan 67, Herrmann 67, Johnson 73, Arnold+Brezzi 85, Comodi 89, Blum+Rannacher 90, Stenberg 91, Krendl+Rafetseder+Zulehner 16, Chen+Hu+Huang 16, Braess+Pechstein+Schöberl 17].

We rewrite the fourth order problem into a second order mixed saddle point problem by intoducing the linearized bending tensor $\sigma=\mathbb{D}\nabla^2 w$ in the $H(\mathrm{div div})=\{\sigma\in L^2(\Omega,\mathbb{R}^{2\times 2}_{\mathrm{sym}})\,|\, \mathrm{div div}\sigma \in H^{-1}(\Omega)\}$ space as additional unknown leading to
\begin{align*}
&\int_{\Omega} \mathbb{D}^{-1}\sigma:\delta\sigma\,dx - \langle \delta\sigma,\nabla^2 w\rangle& &= 0 \qquad \forall \delta\sigma\in H(\mathrm{div div}),\\
& -\langle \sigma,\nabla^2\delta w\rangle& &= -\int_{\Omega}f\,\delta w\,dx \qquad \forall \delta w\in H^1,
 \end{align*}
where $\mathbb{D}^{-1}\varepsilon = \frac{12(1-\nu^2)}{Et^3}\big(\frac{1}{1-\nu}\varepsilon-\frac{\nu}{1-\nu^2}\mathrm{tr}(\varepsilon)I_{2\times2}\big)$ and the duality pairing $\langle\sigma,\nabla^2 w\rangle$ is defined on a triangulation $\mathcal{T}$ of $\Omega$ by
\begin{align*}
\langle\sigma,\nabla^2 w\rangle &= \sum_{T\in\mathcal{T}}\int_{T}\sigma:\nabla^2 w\,dx-\int_{\partial T}\sigma_{nn}\frac{\partial w}{\partial n}\,ds\\
&=-\sum_{T\in\mathcal{T}}\int_{T}\mathrm{div}(\sigma)\cdot\nabla w\,dx+\int_{\partial T}\sigma_{nt}\frac{\partial w}{\partial t}\,ds = -\langle\mathrm{div}\sigma,\nabla w\rangle.
\end{align*}
Here $\partial T$ denotes the element-boundary (edges) of $T$, $n$ the normal vector and $t$ the tangential vector on $\partial T$. With e.g. $\sigma_{nt}:= t^\top\sigma n$ we denote the normal tangential component of $\sigma$.

As a first example, we consider a clamped square plate. Therefore, we use $w\in H^1_0(\Omega)$ to fix the vertical deflection at the boundary together with homogeneous Neumann data for $\sigma$ to fully clamp it. The essential Dirichlet data is the normal-normal component of $\sigma$, $\sigma_{nn}$, which is continuous over elements. Setting $\sigma_{nn}=0$ corresponds to prescribing no stress condition on the boundary being part of the free boundary conditions. Setting $w\in H^1_0(\Omega)$ together with $\sigma_{nn}=0$ gives a simply-supported plate. Setting $w\in H^1(\Omega)$ with homogeneous Neumann data for $\sigma$, the displacement is free, but the plate cannot ''rotate'' at the boundary. 

In [ ]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import unit_square

mesh = Mesh(unit_square.GenerateMesh(maxh=0.3))

In [ ]:
order = 1

V = HDivDiv(mesh, order=order - 1)
Q = H1(mesh, order=order, dirichlet="left|bottom|top|right")
X = V * Q
(sigma, w), (tau, v) = X.TnT()

n = specialcf.normal(2)
def tang(u):
    return u - (u * n) * n

a = BilinearForm(X, symmetric=True)
a += (
    InnerProduct(sigma, tau) + div(sigma) * grad(v) + div(tau) * grad(w) - 1e-10 * w * v
) * dx + (-(sigma * n) * tang(grad(v)) - (tau * n) * tang(grad(w))) * dx(
    element_boundary=True
)
a.Assemble()

f = LinearForm(X)
f += 100 * v * dx
f.Assemble()

gfsol = GridFunction(X)
gfsol.vec.data = a.mat.Inverse(X.FreeDofs(), inverse="sparsecholesky") * f.vec

gfsigma, gfw = gfsol.components

Draw(Norm(gfsigma), mesh, name="sigma")
Draw(gfw, mesh, name="disp", deformation=True);

Next, we consider free boundary conditions at the top and bottom, a simply supported boundary condition at the right, and clamped at the left.

In [ ]:
order = 4
# Free bc at bottom|top, simply-supported at right, clamped at left
V = HDivDiv(mesh, order=order - 1, dirichlet="right|bottom|top")
Q = H1(mesh, order=order, dirichlet="left|right")
X = V * Q
(sigma, w), (tau, v) = X.TnT()

a = BilinearForm(X, symmetric=True)
a += (
    InnerProduct(sigma, tau) + div(sigma) * grad(v) + div(tau) * grad(w) - 1e-10 * w * v
) * dx + (-(sigma * n) * tang(grad(v)) - (tau * n) * tang(grad(w))) * dx(
    element_boundary=True
)
a.Assemble()

f = LinearForm(X)
f += 100 * v * dx
f.Assemble()

gfsol = GridFunction(X)
gfsol.vec.data = a.mat.Inverse(X.FreeDofs(), inverse="sparsecholesky") * f.vec

gfsigma, gfw = gfsol.components

Draw(Norm(gfsigma), mesh, name="sigma", order=4)
Draw(gfw, mesh, name="disp", deformation=True, order=4);

## Hybridization of Hellan-Herrmann-Johnson method
A drawback of the HHJ method in its current form is that it is a saddle point problem. To regain a positive definite system, we can use hybridization techniques [NGSolve docu - Static condensation](https://docu.ngsolve.org/latest/i-tutorials/unit-1.4-staticcond/staticcond.html#1.4-Static-Condensation) by breaking the normal-normal continuity of $\sigma$ and reinforcing it using a normal-continuous Lagrange multiplier $\alpha\in\Lambda$

\begin{align*}
&\int_{\Omega} \mathbb{D}^{-1}\sigma:\tau\,dx - \langle \tau,\nabla^2 w\rangle& + \sum_{E\in\mathcal{E}}\int_E[\![\tau_{nn}]\!]\alpha_n \,ds &= 0&& \qquad \forall \tau\in H^{\mathrm{dc}}(\mathrm{div div}),\\
& -\langle \sigma,\nabla^2 v\rangle& &= -\int_{\Omega}f\,v\,dx&& \qquad \forall v\in H^1,\\
&\sum_{E\in\mathcal{E}}\int_E[\![\sigma_{nn}]\!]\delta_n \,ds &&=0&& \qquad \forall v\in \Lambda.

\end{align*}

<center>
<table><tr>
<td> <img src="h1_p1_trig.png" width="150"/> </td>
<td> <img src="hdivdiv_0_trig_2.png" width="170"/> </td>
<td> <img src="normalfacet_0_trig_0.png" width="150"/> </td>
</tr>
</table>
</center>

This enables us to statically condense out $\sigma$ and the resulting system in $(u,\alpha)$ is again symmetric positive definite such that we can use e.g. sparsecholesky solver or CG. The resulting degrees of freedom are equivalent to the famous Morley triangle [<a href="https://doi.org/10.1243/03093247V061020">Morley. The constant-moment plate-bending element. <i> Journal of Strain Analysis 6 </i>, 1 (1971), 20-24</a>] which is a non-conforming element for the fourth order plate problem. The physical meaning of $\alpha$ is the normal derivative of the displacement $\alpha_ n \hat{=}\frac{\partial w}{\partial n}$.

<center>
<img src="morley_element.png" width="150"/>
</center>

Note that the essential and natural boundary conditions from $\sigma$ to $\alpha$ swap.

In [ ]:
order = 1

V = Discontinuous(HDivDiv(mesh, order=order - 1))
Q = H1(mesh, order=order, dirichlet="left|bottom|top|right")
H = NormalFacetFESpace(mesh, order=order - 1, dirichlet="left|bottom|top|right")
X = V * Q * H
(sigma, w, alpha), (tau, v, beta) = X.TnT()

n = specialcf.normal(2)
def tang(u):
    return u - (u * n) * n

a = BilinearForm(X, symmetric=True, condense=True)
a += (
    (InnerProduct(sigma, tau) + div(sigma) * grad(v) + div(tau) * grad(w)) * dx
    - (sigma * n * tang(grad(v)) + tau * n * tang(grad(w))) * dx(element_boundary=True)
    + (sigma[n,n] * beta*n + tau[n,n] * alpha*n) * dx(element_boundary=True)
)
a.Assemble()

f = LinearForm(X)
f += 100 * v * dx
f.Assemble()

gfsol = GridFunction(X)

r = f.vec.CreateVector()
r.data = f.vec - a.mat * gfsol.vec
r.data += a.harmonic_extension_trans * r
gfsol.vec.data += a.mat.Inverse(X.FreeDofs(a.condense), inverse="sparsecholesky") * r
gfsol.vec.data += a.harmonic_extension * gfsol.vec
gfsol.vec.data += a.inner_solve * r

gfsigma, gfw, gfalpha = gfsol.components
Draw(gfw, mesh, name="disp", deformation=True)
Draw(Norm(gfw.Operator("hesse")), mesh, name="sigma(w)")
Draw(Norm(gfsigma), mesh, name="sigma");

## Postprocessing property of HHJ method
We can use lowest order elements, i.e. linear polynomials for the vertical deflection $w\in\Pi^1$ and piece-wise constants for the moment tensor $\sigma$. As $\sigma\,\approx \nabla^2 w$ one might try to compute a new $\tilde w\in \Pi^2$ as a postprocessing step element-wise with an improved order of convergence.


[<a href="https://doi.org/10.1051/m2an/1991250101511">Stenberg. Postprocessing schemes for some mixed finite elements. <i> ESAIM: M2AN 25 </i>, 1 (1991), 151-167</a>]

We solve the following problem element-wise
$$
\widetilde w = \operatorname{arg}\min_{v_h \in P^{k+1} \atop v_h(V) = w_h(V)} \| \nabla^2 v_h -  \sigma \|_{L_2}^2
$$
under the constraint that the vertex values are preserved. Therefore, we use a discontinuous Lagrange space and set on each element the dofs corresponding to a vertex as fixed.

In [ ]:
X2 = Discontinuous(H1(mesh, order=order + 1))
u2, v2 = X2.TnT()

freedofs = BitArray(X2.ndof)
freedofs[:] = True
offset = 3 + int(3 * (order - 1 + 1) + 0.5 * (order - 1 + 1) * (order - 2 + 1))
for i in range(mesh.ne):
    freedofs[i * offset + 0] = False
    freedofs[i * offset + 1] = False
    freedofs[i * offset + 2] = False
# print(freedofs)

a2 = BilinearForm(X2)
a2 += InnerProduct(u2.Operator("hesse"), v2.Operator("hesse")) * dx
f2 = LinearForm(X2)
f2 += InnerProduct(gfsigma, v2.Operator("hesse")) * dx

gfw2 = GridFunction(X2)
# set the vertex data by interpolation
gfw2.Set(gfw, dual=True)

a2.Assemble()
f2.Assemble()

r = gfw2.vec.CreateVector()
r.data = a2.mat * gfw2.vec - f2.vec
gfw2.vec.data -= a2.mat.Inverse(freedofs=freedofs, inverse="sparsecholesky") * r

Draw(gfw2, mesh, "postprocess", deformation=True)
Draw(Norm(gfw2.Operator("hesse")), mesh, name="sigma(w)");

Other postprocessing method: [<a href="https://doi.org/10.1007/s10915-021-01595-9">Li. Recovery-based a posteriori error analysis for plate bending problems. <i>J Sci Comput.</i> 2021</a>]

Correct displacement $w \in V_h^k$ with bubble functions of one degree higher $w_b\in V_h^{k+1}$ such that $\tilde{w} = w+w_b$

\begin{align*}
\widetilde w = \operatorname{arg}\min_{v_h \in V_h^{k+1} \atop v_h|_{V_h^k} = w_h} \| \mathbb{D}\nabla^2 v_h -  \sigma \|_{L_2}^2.
\end{align*}

We need to solve a global problem. However, it was proved that the condition number of a Jacobi preconditioner stays bounded. Therefore, the costs are comparable to solve local problems.

In [ ]:
from ngsolve.krylovspace import CGSolver

print("order=", order)
X = H1(mesh, order=order + 1, dirichlet=".*")
w, dw = X.TnT()

# lock dofs
freedofs = BitArray([False] * X.ndof)
for el in mesh.edges:
    freedofs[X.GetDofNrs(el)[order - 1]] = True
freedofs &= ~X.GetDofs(mesh.Boundaries(".*"))

a = BilinearForm(X, symmetric=True)
a += InnerProduct(w.Operator("hesse"), dw.Operator("hesse"))*dx

f = LinearForm(X)
f += InnerProduct(gfsigma - gfw.Operator("hesse"), dw.Operator("hesse"))*dx

gfw3 = GridFunction(X)

a.Assemble()
f.Assemble()

preJpoint = a.mat.CreateSmoother(freedofs)
a.Assemble()

inv = CGSolver(a.mat, pre=preJpoint, printrates="\r", maxiter=800)
gfw3.vec.data = inv * f.vec  

Draw(gfw3, mesh, "w3", order=3)
Draw(gfw, mesh, "w3", order=3, deformation=True)
Draw(gfw + gfw3, mesh, "w3", order=3, deformation=True);